In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Installs
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet
!pip install tensorflow-text==2.15.0 --quiet
!pip install transformers==4.17 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.68 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.3 MB/s eta 0:00:00


In [ ]:
#@title Imports

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
#import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import transformers

from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [ ]:
#@title Plotting Function

# 4-window plot. Small modification from matplotlib examples.

def make_plot(axs,
              model_history1,
              model_history2,
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        # small adjustment to account for the 2 accuracy measures in the Weighted Averging Model with Attention
        if 'classification_%s' % metric in model_history2.history:
            metric2 = 'classification_%s' % metric
        else:
            metric2 = metric

        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric2])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric2])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            model_metric = [metric, metric2][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[model_metric])
            ax1.plot(model_history.history['val_%s' % model_metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

#Read Data

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

In [ ]:
train_filepath = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_training_data.csv'
test_filepath = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_testing_data.csv'

In [ ]:
train_dataset = pd.read_csv(train_filepath)
test_dataset = pd.read_csv(test_filepath)
num_train_examples = len(train_dataset)

In [ ]:
#stream dataset
#train_dataset = load_dataset('csv', data_files = train_filepath, streaming = True)['train']
#test_dataset = load_dataset('csv', data_files = test_filepath, streaming = True)['train']

NameError: name 'load_dataset' is not defined

In [ ]:
X_train = train_dataset['doc_text']
y_train = train_dataset['label']
X_test = test_dataset['doc_text']
y_test = test_dataset['label']

In [ ]:
#@title BERT Tokenization of training and test data
MAX_SEQUENCE_LENGTH = 100


train_examples_str = [X for X in X_train]
test_examples_str = [X for X in X_test]


bert_train_tokenized = bert_tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]

bert_train_labels = np.array(y_train)

bert_test_tokenized = bert_tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_test_inputs = [bert_test_tokenized.input_ids,
                     bert_test_tokenized.token_type_ids,
                     bert_test_tokenized.attention_mask]
bert_test_labels = np.array(y_test)


In [ ]:
def create_bert_cls_model(bert_base_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.000025,
                          trainable = True):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """

    ### YOUR CODE HERE
    bert_model = TFBertModel.from_pretrained(f'{bert_base_model}')

    #ensures bert model is fully trainable
    if trainable == True:
      bert_model.trainable = True
    else:
      #bert_model.trainable = False
      for param in bert_model.bert.embeddings.weights:
        param._trainable = False
      for param in bert_model.bert.encoder.layer[0:6]:
        param._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    #pooler_token = bert_out[1]
    cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics=['accuracy', keras.metrics.Recall()])

    ### END YOUR CODE

    return classification_model

In [ ]:
### YOUR CODE HERE
early_stopping = EarlyStopping(monitor="val_accuracy", min_delta= 0.015, patience=2, restore_best_weights=True)

bert_classification_model = create_bert_cls_model('bert-base-uncased', hidden_size = 150, trainable = False) #partial forzen weights
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    # [bert_train_inputs[0][:80], bert_train_inputs[1][:80], bert_train_inputs[2][:80]],
    # bert_train_labels[0:80],
    # validation_data = ([bert_test_inputs[0][:20], bert_test_inputs[1][:20], bert_test_inputs[2][:20]], bert_test_labels[0:20]),
    batch_size = 8,
    epochs = 10,
    callbacks = [early_stopping]
)

### END YOUR CODE

Epoch 1/10


790/790 [==============================] - 145s 150ms/step - loss: 0.6728 - accuracy: 0.5875 - recall_9: 0.6576 - val_loss: 0.6420 - val_accuracy: 0.6219 - val_recall_9: 0.7987
Epoch 2/10
790/790 [==============================] - 122s 154ms/step - loss: 0.6092 - accuracy: 0.6668 - recall_9: 0.7333 - val_loss: 0.6368 - val_accuracy: 0.6491 - val_recall_9: 0.7252
Epoch 3/10
790/790 [==============================] - 122s 154ms/step - loss: 0.5158 - accuracy: 0.7531 - recall_9: 0.8093 - val_loss: 0.6580 - val_accuracy: 0.6624 - val_recall_9: 0.6877
Epoch 4/10
790/790 [==============================] - 122s 155ms/step - loss: 0.3843 - accuracy: 0.8326 - recall_9: 0.8657 - val_loss: 0.8609 - val_accuracy: 0.6529 - val_recall_9: 0.7600


In [ ]:
### YOUR CODE HERE
bert_classification_model = create_bert_cls_model('bert-base-cased', trainable = True) #overfitting
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 6
)

### END YOUR CODE

Epoch 1/6


790/790 [==============================] - 222s 228ms/step - loss: 0.6834 - accuracy: 0.5724 - val_loss: 0.7062 - val_accuracy: 0.5187
Epoch 2/6
790/790 [==============================] - 179s 227ms/step - loss: 0.6226 - accuracy: 0.6656 - val_loss: 0.6429 - val_accuracy: 0.6510
Epoch 3/6
790/790 [==============================] - 179s 226ms/step - loss: 0.5236 - accuracy: 0.7588 - val_loss: 0.6307 - val_accuracy: 0.6586
Epoch 4/6
790/790 [==============================] - 180s 228ms/step - loss: 0.3755 - accuracy: 0.8451 - val_loss: 0.7061 - val_accuracy: 0.6700
Epoch 5/6
790/790 [==============================] - 179s 226ms/step - loss: 0.2729 - accuracy: 0.8906 - val_loss: 0.8243 - val_accuracy: 0.6409
Epoch 6/6
790/790 [==============================] - 179s 226ms/step - loss: 0.1742 - accuracy: 0.9341 - val_loss: 1.4979 - val_accuracy: 0.6662


### Add in early stopping above; (patience at 2 epochs) Try Frozen BERT + CNN

In [ ]:
early_stopping = EarlyStopping(monitor="val_accuracy", min_delta= 0.025, patience=1, restore_best_weights=True)

### YOUR CODE HERE
bert_classification_model = create_bert_cls_model('bert-base-cased', hidden_size = 75, trainable = True) #overfitting; trying with less hidden layer size; best performance with least overfitting
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 6,
    callbacks = [early_stopping]
)

### END YOUR CODE

Epoch 1/6


749/749 [==============================] - 268s 304ms/step - loss: 0.7004 - accuracy: 0.5169 - recall_2: 0.4935 - val_loss: 0.6958 - val_accuracy: 0.4061 - val_recall_2: 0.7086
Epoch 2/6
749/749 [==============================] - 225s 301ms/step - loss: 0.6846 - accuracy: 0.5526 - recall_2: 0.5904 - val_loss: 0.7664 - val_accuracy: 0.3283 - val_recall_2: 0.8142


In [ ]:
### YOUR CODE HERE
bert_classification_model = create_bert_cls_model('bert-base-cased', hidden_size = 25, trainable = True) #unfreezing weights; try toning down hidden layers size
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 6
)

### END YOUR CODE

Epoch 1/6


790/790 [==============================] - 223s 232ms/step - loss: 0.6809 - accuracy: 0.5758 - val_loss: 0.6654 - val_accuracy: 0.6111
Epoch 2/6
790/790 [==============================] - 175s 221ms/step - loss: 0.6423 - accuracy: 0.6454 - val_loss: 0.6392 - val_accuracy: 0.6396
Epoch 3/6
790/790 [==============================] - 174s 220ms/step - loss: 0.5607 - accuracy: 0.7344 - val_loss: 0.6549 - val_accuracy: 0.6574
Epoch 4/6
790/790 [==============================] - 166s 210ms/step - loss: 0.4322 - accuracy: 0.8162 - val_loss: 0.6713 - val_accuracy: 0.6681
Epoch 5/6
790/790 [==============================] - 173s 219ms/step - loss: 0.3165 - accuracy: 0.8792 - val_loss: 0.8088 - val_accuracy: 0.6624
Epoch 6/6
790/790 [==============================] - 174s 220ms/step - loss: 0.2277 - accuracy: 0.9197 - val_loss: 0.8688 - val_accuracy: 0.6517


In [ ]:
### YOUR CODE HERE
bert_classification_model = create_bert_cls_model('bert-base-cased', hidden_size = 15, trainable = True) #no progress @ hidden size = 15
bert_classification_model_history = bert_classification_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data = (bert_test_inputs, bert_test_labels),
    batch_size = 8,
    epochs = 6
)

### END YOUR CODE

Epoch 1/6


790/790 [==============================] - 221s 224ms/step - loss: 0.6976 - accuracy: 0.5268 - val_loss: 0.6931 - val_accuracy: 0.4908
Epoch 2/6
790/790 [==============================] - 170s 215ms/step - loss: 0.6951 - accuracy: 0.5096 - val_loss: 0.6934 - val_accuracy: 0.4908
Epoch 3/6
790/790 [==============================] - 167s 211ms/step - loss: 0.6932 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.4908
Epoch 4/6
790/790 [==============================] - 165s 209ms/step - loss: 0.6931 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.4908
Epoch 5/6
790/790 [==============================] - 165s 209ms/step - loss: 0.6931 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.4908
Epoch 6/6
790/790 [==============================] - 166s 210ms/step - loss: 0.6931 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.4908
